In [22]:
def check_item(feats, key, el):
    if key in feats:
        return el in feats[key]
    return False

# c(domains).verb(nsubj='c_pron', obj='c_noun')
def verb_pattern_checker(domains):
    rel_feats={x[0]:x[5] for x in domains}
    if(check_item(rel_feats, 'nsubj', 'c_pron') and
       check_item(rel_feats, 'obj', 'c_noun')):
        print('pattern: verb+nsubj(pron)+obj(noun)')
        
class Patterns(object):
    _name = None
    _fields = {}  # {field: field object}
    def __init__(self, domains=None, track=True):
        super(Patterns, self).__init__()
        self.domains=domains
        self.track=track
    def __getattr__(self, method):
        """Provide a dynamic access to a service method."""
        if method.startswith('_'):
            return super(Patterns, self).__getattr__(method)

        def service_method(*args, **kwargs):
            """Return the result of the check request."""
            rel_feats={x[0]:x[5] for x in self.domains}
            options=[]                
            result=True
            for key, value in kwargs.items():
                opt_ret=check_item(rel_feats, key, value)
                if not opt_ret:
                    result=False
                options.append('{} is {}: {}'.format(key,value,opt_ret))
            if self.track:
                return "%s with %s"%(method, ', '.join(options)), result
            else:
                return result
        return service_method

In [25]:
def print_result(rs):
    for r in rs:
        print('%s [%s]'%('✔' if r[1] else '✖', r[0]))
def verb_patterns(domains):
    print_result([Patterns(domains).verb(nsubj='c_pron', obj='c_noun'),
           Patterns(domains).verb(nsubj='c_noun', obj='c_noun')])
                
def get_verb_domains(data, return_df=False):
    import requests
    import sagas
    response = requests.post('http://localhost:8090/verb_domains', json=data)
    # print(response.status_code, response.json())
    df_set=[]
    if response.status_code == 200:
        rs = response.json()
        for r in rs:
            type_name=r['type']

            if type_name=='verb_domains':
                print('✧', r['verb'], r['index'])
                # verb_pattern_checker(r['domains'])
                domains=r['domains']
                verb_patterns(domains)
            elif type_name=='aux_domains':
                # 'rel': word.dependency_relation, 'governor': word.governor, 'head': dc.text
                print('℗', r['aux'], r['rel'], '☇', r['head'])
            elif type_name=='subj_domains':
                print('✁', r['subj'], r['rel'], '☇', r['head'])
            else:
                raise Exception('Cannot process specific type: {}'.format(type_name))

            df = sagas.to_df(r['domains'], ['rel', 'index', 'text', 'lemma', 'children', 'features'])
            df_set.append(df)
            if not return_df:
                sagas.print_df(df)
    if return_df:
        return df_set

source='de'
text='Du wirst das Fenster kaputt machen.'
data = {'lang': source, "sents": text}
for df in get_verb_domains(data, True):
    display(df)

✧ machen 6
✔ [verb with nsubj is c_pron: True, obj is c_noun: True]
✖ [verb with nsubj is c_noun: False, obj is c_noun: True]


,rel,index,text,lemma,children,features
0,nsubj,1,Du,du,[Du],[c_pron]
1,aux,2,wirst,werden,[wirst],[c_aux]
2,obj,4,Fenster,Fenster,"[das, Fenster]",[c_noun]
3,xcomp,5,kaputt,kaputt,[kaputt],[c_adv]
4,punct,7,.,.,[.],[c_punct]
